In [1]:
"""Converts MNIST data to TFRecords file format with Example protos."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import os
import time
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import mnist
# mnist is a prewriten py file containing the definition of the graph
print(dir(mnist))

TRANFILE = 'data/train.tfrecords'
TESTFILE = 'data/test.tfrecords'
VALIDATIONFILe=  'data/validation.tfrecords'

['IMAGE_PIXELS', 'IMAGE_SIZE', 'NUM_CLASSES', '__builtins__', '__doc__', '__file__', '__name__', '__package__', 'absolute_import', 'division', 'evaluation', 'inference', 'loss', 'math', 'print_function', 'tf', 'training']


In [2]:
def read_and_decode(filename_queue):
    reader = tf.TFRecordReader()
    _, serialized_example = reader.read(filename_queue)
    features = tf.parse_single_example(
      serialized_example,
      # Defaults are not specified since both keys are required.
      features={
          'label': tf.FixedLenFeature([], tf.int64),
          'image_raw': tf.FixedLenFeature([], tf.string)
            
      })

    # Convert from a scalar string tensor (whose single string has
    # length mnist.IMAGE_PIXELS) to a uint8 tensor with shape
    # [mnist.IMAGE_PIXELS].
    image = tf.decode_raw(features['image_raw'], tf.float32)
    image.set_shape([mnist.IMAGE_PIXELS])

    # OPTIONAL: Could reshape into a 28x28 image and apply distortions
    # here.  Since we are not applying any distortions in this
    # example, and the next step expects the image to be flattened
    # into a vector, we don't bother.

    # Convert from [0, 255] -> [-0.5, 0.5] floats.
    image = image - 0.5

    # Convert label from a scalar uint8 tensor to an int32 scalar.
    label = tf.cast(features['label'], tf.int32)

    return image, label


def inputs(filename, batch_size, num_epochs):
    """Reads input data num_epochs times.
    Args:
      filename: filename of TFRecords
      batch_size: Number of examples per returned batch.
      num_epochs: Number of times to read the input data, or 0/None to
      train forever.
    Returns:
        A tuple (images, labels), where:
        * images is a float tensor with shape [batch_size, mnist.IMAGE_PIXELS]
        in the range [-0.5, 0.5].
        * labels is an int32 tensor with shape [batch_size] with the true label,
        a number in the range [0, mnist.NUM_CLASSES).
    Note that an tf.train.QueueRunner is added to the graph, which
    must be run using e.g. tf.train.start_queue_runners().
    """
    if not num_epochs: num_epochs = None
     

    with tf.name_scope('input'):
        filename_queue = tf.train.string_input_producer(
            [filename], num_epochs=num_epochs)

        # Even when reading in multiple threads, share the filename
        # queue.
        image, label = read_and_decode(filename_queue)

        # Shuffle the examples and collect them into batch_size batches.
        # (Internally uses a RandomShuffleQueue.)
        # We run this in two threads to avoid being a bottleneck.
        images, sparse_labels = tf.train.shuffle_batch(
            [image, label], batch_size=batch_size, num_threads=2,
            capacity=1000 + 3 * batch_size,
            # Ensures a minimum amount of shuffling of examples.
            min_after_dequeue=1000)

    return images, sparse_labels



    

   

In [ ]:
# Train MNIST for a number of steps and save the session to model/.

# Tell TensorFlow that the model will be built into the default Graph.
with tf.Graph().as_default(): # define graph
    # Input images and labels from the training set
    images, labels = inputs('data/train.tfrecords', batch_size=100,
                                num_epochs=20)

    # Build a Graph that computes predictions from the inference model.
    logits = mnist.inference(images,
                                 128,# number of neurons at hiddenlayer 1
                                 32) # number of neurons at hiddenlayer 2

    # Add to the Graph the loss calculation.
    loss = mnist.loss(logits, labels)

    # Add to the Graph operations that train the model.
    train_op = mnist.training(loss, learning_rate=0.01)

    # The op for initializing the variables.
    init_op = tf.group(tf.initialize_all_variables(),
                        tf.initialize_local_variables())
    # Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    # Create a session for running operations in the Graph.
    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4))
    # by default, tf.Session has 2 threads, 
    # you can use config=tf.ConfigProto(intra_op_parallelism_threads=NUM_THREADS) 
    # to specify the number of threads

    # Initialize the variables (the trained variables and the 
    # epoch counter).
    sess.run(init_op)

    _, loss_value = sess.run([train_op, loss])
     
    duration = time.time() - start_time
    save_path = saver.save(sess, 'model_parameter/mnist_128_32.cpkt')
    sess.close()

print(duration)

# Train MNIST for a number of steps and save the session to model/.

# Tell TensorFlow that the model will be built into the default Graph.
## the following code is to  use multi-threads
with tf.Graph().as_default(): # define graph
    # Input images and labels from the training set
    images, labels = inputs('data/train.tfrecords', batch_size=100,
                                num_epochs=20)

    ### Build a Graph that computes predictions from the inference model.
    logits = mnist.inference(images,
                                 128,# number of neurons at hiddenlayer 1
                                 32) # number of neurons at hiddenlayer 2

    ### Add to the Graph the loss calculation.
    loss = mnist.loss(logits, labels)

    ### Add to the Graph operations that train the model.
    train_op = mnist.training(loss, learning_rate=0.01)

    ### The op for initializing the variables.
    init_op = tf.group(tf.initialize_all_variables(),
                        tf.initialize_local_variables())
    ### Add ops to save and restore all the variables.
    saver = tf.train.Saver()
    ### Create a session for running operations in the Graph.
    sess = tf.Session(config=tf.ConfigProto(intra_op_parallelism_threads=4))
    ### by default, tf.Session has 2 threads, 
    ### you can use config=tf.ConfigProto(intra_op_parallelism_threads=NUM_THREADS) 
    ### to specify the number of threads

    ### Initialize the variables (the trained variables and the 
    ### epoch counter).
    sess.run(init_op)

    ### Start input enqueue threads.
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    ####
    start_time = time.time()

    try:
        step = 0
        total_duration = 0
        while not coord.should_stop():
            

            # Run one step of the model.  The return values are
            # the activations from the `train_op` (which is
            # discarded) and the `loss` op.  To inspect the values
            # of your ops or variables, you may include them in
            # the list passed to sess.run() and the value tensors
            # will be returned in the tuple from the call.
            _, loss_value = sess.run([train_op, loss])

            
            # total_duration+=duration

            # Print an overview fairly often.
            if step % 1000 == 0:
                print('Step %d: loss = %.2f ' % (step, loss_value,))
            step += 1
    except tf.errors.OutOfRangeError:
        print('Done training for %d epochs, %d steps.' % (2, step))
    finally:
        # When done, ask the threads to stop.
        coord.request_stop()

        ### Wait for threads to finish.
    coord.join(threads)
    duration = time.time() - start_time
    save_path = saver.save(sess, 'model_parameter/mnist_128_32.cpkt')
    sess.close()

print(duration)

In [4]:
print(duration)

168.582716942


In [9]:
print(total_duration)

173.698046684


# build the graph and reload the train variables
all the trainable variable is in the collection tf.trainable_variables(), we only load those varialbes to the model.

In [106]:
with tf.Graph().as_default(): 
    images, labels = inputs('data/test.tfrecords', batch_size=100,
                                num_epochs=1)

    
    logits = mnist.inference(images,
                                 128,# number of neurons at hiddenlayer 1
                                 32) # number of neurons at hiddenlayer 2
    correct = tf.nn.in_top_k(logits, labels, 1)
    predict_right = tf.reduce_sum(tf.cast(correct, tf.int32))
    
    saver = tf.train.Saver(tf.trainable_variables())
    init_op = tf.group(tf.initialize_all_variables(),
                        tf.initialize_local_variables())
    
    with tf.Session() as sess:
        sess.run(init_op)
        saver.restore(sess,'model_parameter/mnist_128_32.cpkt')
        coord = tf.train.Coordinator()
        threads = tf.train.start_queue_runners(sess=sess, coord=coord)
    #

        try:
            step = 0
            total_right = 0
            total_example = 0
            while not coord.should_stop():
                start_time = time.time()

                
                right_case = predict_right.eval()
                duration = time.time() - start_time

                # Print an overview fairly often.
                if step % 10 == 0:
                    print('Step %d: predict_right = %.2f (%.3f sec)' % (step, right_case,
                                                     duration))
                
                
                step += 1
                total_right += right_case
                total_example += int(images.get_shape()[0])
        except tf.errors.OutOfRangeError:
            print('Done training for %d epochs, %d steps.' % (2, step))
        finally:
            # When done, ask the threads to stop.
            coord.request_stop()

        # Wait for threads to finish.
        coord.join(threads)
        print('accuracy of the model on test is %.2f' %(total_right*1.0/total_example))  
        print('%d,%d' %(total_right,total_example))
    

Step 0: predict_right = 94.00 (0.128 sec)
Step 10: predict_right = 92.00 (0.012 sec)
Step 20: predict_right = 92.00 (0.012 sec)
Step 30: predict_right = 96.00 (0.012 sec)
Step 40: predict_right = 93.00 (0.013 sec)
Step 50: predict_right = 95.00 (0.012 sec)
Step 60: predict_right = 97.00 (0.013 sec)
Step 70: predict_right = 91.00 (0.015 sec)
Step 80: predict_right = 98.00 (0.013 sec)
Step 90: predict_right = 97.00 (0.003 sec)
Done training for 2 epochs, 100 steps.
accuracy of the model on test is 0.94
9401,10000


### however, it is not convenient to gather images when using TFRecords and multiple threads 
### due to the map-reduce principles.